In [19]:
import multiprocessing
import uuid
import os

if not os.path.exists('out'):
    os.makedirs('out')
if not os.path.exists('results'):
    os.makedirs('results')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs.latin_design import LatinDesign
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction

In [20]:
target_name = 'SEIZ_updated'
threads_per_run = multiprocessing.cpu_count()
init_exposures = 100

parameter_space = ParameterSpace([ContinuousParameter('p', 0, 1),
                                  ContinuousParameter('l', 0, 1),
                                  ContinuousParameter('epsilon', 2, 2),
                                  DiscreteParameter('prop_init_skeptics', np.arange(1, init_exposures-1, 1, dtype=np.int_))
                                  # DiscreteParameter('prop_init_skeptics', np.array([50]))
                                ])

In [21]:
def run_job(p, l, epsilon, init_skeptics, init_adopters):
    with open(f'{target_name}_editable.fred', 'r') as file:
        filedata = file.read()

    vars = {
        'p': p, 
        'l': l, 
        'epsilon': epsilon, 
        'init_skeptic': int(init_skeptics), 
        'init_adopt': int(init_adopters)
    }

    run_uuid = uuid.uuid4()
    var_str = ', '.join(map(lambda item: f'{item[0]}={item[1]}', vars.items()))
    print(run_uuid, var_str)

    with open(f'out/{target_name}_{run_uuid}_out.fred', 'w') as file:
        file.write(filedata.format(**vars))
        
    !fred_job -p out/{target_name}_{run_uuid}_out.fred -k {target_name}_{run_uuid}_run -t {threads_per_run}
    !fred_csv -k {target_name}_{run_uuid}_run > results/{target_name}_{run_uuid}_run.csv

    with open(f'results/{target_name}_{run_uuid}_run.csv', 'r') as file:
        lines = file.readlines()

    with open(f'results/{target_name}_{run_uuid}_run.csv', 'w') as file:
        lines[0] = var_str+'\n'
        file.writelines(lines)

    return pd.read_csv(f'results/{target_name}_{run_uuid}_run.csv', header=2)

In [22]:
def sample(X, clear_results=False):
    X = np.array(X)
    
    if X.ndim < 2:
        X = np.expand_dims(X, 0)
    elif X.ndim > 2:
        raise Exception(f'X has too many dimensions (ndim={X.ndim}, must be 1 or 2)')
    if X.shape[-1] != len(parameter_space._parameters):
        raise Exception(f'X has the wrong number of variables (variables={X.shape[-1]}, must be {len(parameter_space._parameters)})')

    X = np.append(X, np.expand_dims(init_exposures-X[:,-1], axis=1), axis=1)  # create and add init_adopters variable

    if clear_results:
        !yes | fred_clear_all_results
    
    Y = []

    with multiprocessing.Pool() as pool:
        for data in pool.starmap(run_job, X):
            S_count = data['ADOPT.totS'].iat[0]
            Y.append([(-data['ADOPT.S'].diff()).max() / S_count])
        
    return np.vstack(Y)

In [23]:
design = LatinDesign(parameter_space)
num_data_points = 10
X = design.get_samples(num_data_points)

In [24]:
X

array([[2.5e-01, 4.5e-01, 2.0e+00, 9.3e+01],
       [4.5e-01, 2.5e-01, 2.0e+00, 6.4e+01],
       [9.5e-01, 3.5e-01, 2.0e+00, 6.0e+00],
       [1.5e-01, 1.5e-01, 2.0e+00, 1.6e+01],
       [7.5e-01, 5.5e-01, 2.0e+00, 5.4e+01],
       [6.5e-01, 6.5e-01, 2.0e+00, 4.5e+01],
       [3.5e-01, 9.5e-01, 2.0e+00, 2.5e+01],
       [8.5e-01, 7.5e-01, 2.0e+00, 8.3e+01],
       [5.5e-01, 5.0e-02, 2.0e+00, 3.5e+01],
       [5.0e-02, 8.5e-01, 2.0e+00, 7.4e+01]])

In [25]:
Y = sample(X, clear_results=True)

You are about to delete /home/nyoma/FRED/RESULTS. This cannot be undone.
Proceed? yes/no [no]
/home/nyoma/FRED/RESULTS deleted
yes: standard output: Broken pipe
d7d45ed5-70f7-4552-910b-1101af8b3cacf4107796-ccc6-49e3-be2a-38ff149b691706ac704d-5f13-4dd5-a2df-d876a9fb9fa0f8a393a8-5410-480f-a135-c3721f8417782ac16b47-3f6a-47bb-ae32-d00929d818f42392171c-cba9-410a-bb50-d706eea195ad79635772-3444-456e-8914-2184a9d4899b30728e6c-8e8c-478d-be88-178d20c6fde82ea7a272-2f68-4a60-b62a-863f84dd23ecd739d7d8-2c92-4f25-86a2-bd6794ac593b          p=0.45, l=0.25, epsilon=2.0, init_skeptic=64, init_adopt=36p=0.15000000000000002, l=0.15000000000000002, epsilon=2.0, init_skeptic=16, init_adopt=84p=0.35000000000000003, l=0.95, epsilon=2.0, init_skeptic=25, init_adopt=75p=0.95, l=0.35000000000000003, epsilon=2.0, init_skeptic=6, init_adopt=94p=0.25, l=0.45, epsilon=2.0, init_skeptic=93, init_adopt=7p=0.6500000000000001, l=0.6500000000000001, epsilon=2.0, init_skeptic=45, init_adopt=55p=0.75, l=0.55, epsilon=2.0, 


KeyboardInterrupt



In [ ]:
Y

In [ ]:
model_gpy = GPRegression(X,Y)
model_emukit = GPyModelWrapper(model_gpy)

In [ ]:
model_variance = ModelVariance(model=model_emukit)
expdesign_loop = ExperimentalDesignLoop(model=model_emukit,
                                        space=parameter_space,
                                        acquisition=model_variance,
                                        batch_size=5)

max_iterations = 50
expdesign_loop.run_loop(sample, max_iterations)

In [ ]:
model_emukit.predict(np.array([[0.5,0.5,2,50]]))